In [3]:
import zipfile
import pandas as pd

In [4]:
# Percorso del file ZIP da estrarre
zip_mob_file_path = "alarms datasets/mob/20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.zip"
zip_tx_adsl_file_path = "alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.zip"
zip_tx_pdh_file_path = "alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.zip"
zip_tx_ptn_file_path = "alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.zip"

# Percorso della cartella di destinazione per l'estrazione
extract_to_folder_mob = "alarms datasets/mob"
extract_to_folder_tx = "alarms datasets/tx"

# Percorso del file ZIP da estrarre
zip_files_paths = {
    zip_mob_file_path: extract_to_folder_mob,
    zip_tx_adsl_file_path: extract_to_folder_tx,
    zip_tx_pdh_file_path: extract_to_folder_tx,
    zip_tx_ptn_file_path: extract_to_folder_tx,
}

# Aprire il file ZIP
for zip_file_path, extract_to_folder in zip_files_paths.items():
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Estrarre tutto il contenuto nella cartella di destinazione
        zip_ref.extractall(extract_to_folder)

In [5]:
mob_alarms_df = pd.read_parquet('alarms datasets/mob/20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet')

tx_alarms_adsl_df = pd.read_parquet('alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet')
tx_alarms_pdh_df = pd.read_parquet('alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet')
tx_alarms_ptn_df = pd.read_parquet('alarms datasets/tx/20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet')

In [13]:
tx_alarms_df = pd.concat([tx_alarms_adsl_df, tx_alarms_pdh_df, tx_alarms_ptn_df])

In [6]:
mob_alarms_df


,alarm_id,data,first_occurrence,first_severity,last_occurrence,last_update,loc_descr,make_prediction,ne_id,network,...,is_ack_or_notes,is_tt,is_abam_tt,loc_name,tt_id,link_id,loc_name_a,loc_name_z,lp_id,lp_type
0,EFM;63b098b01a65d200011620f8,2023-01-01 00:00:02,2022-12-31 21:16:48,critical,2022-12-31 23:37:41,2022-12-31 23:58:25,<NA>,False,etr100009,an,...,False,False,False,NaN,<NA>,NaN,None,None,None,None
1,EFM;63b0b476c623b40001a68b45,2023-01-01 00:00:02,2022-12-31 23:15:18,major,2022-12-31 23:20:09,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,False,False,False,ceseitej,<NA>,cxa2pc7,None,None,None,None
2,EFM;63b0b5a8c623b40001a68b9b,2023-01-01 00:00:02,2022-12-31 23:20:24,major,2022-12-31 23:20:49,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,False,False,False,ceseitej,<NA>,cxa2pc7,None,None,None,None
3,EFM;63b0b5c6c623b40001a68ba2,2023-01-01 00:00:02,2022-12-31 23:20:54,major,2022-12-31 23:23:09,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,False,False,False,ceseitej,<NA>,cxa2pc7,None,None,None,None
4,EFM;63b0b675c623b40001a68c03,2023-01-01 00:00:02,2022-12-31 23:23:49,major,2022-12-31 23:32:15,2022-12-31 23:58:25,fot127 quarto sud mc,False,gbtsfo127,an,...,False,False,False,ceseitej,<NA>,cxa2pc7,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28176439,EFM;6591f1f639628d000137853b,2024-01-01 00:00:03,2023-12-31 23:57:58,major,2023-12-31 23:59:42,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,False,False,False,NaN,<NA>,NaN,None,None,None,None
28176440,EFM;6591f1f739628d000137854e,2024-01-01 00:00:03,2023-12-31 23:57:58,major,2023-12-31 23:59:52,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,False,False,False,NaN,<NA>,NaN,None,None,None,None
28176441,EFM;6591f1f739628d0001378554,2024-01-01 00:00:03,2023-12-31 23:57:58,major,2023-12-31 23:59:52,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,False,False,False,NaN,<NA>,NaN,None,None,None,None
28176442,EFM;6591f1f939628d0001378585,2024-01-01 00:00:03,2023-12-31 23:57:59,major,2023-12-31 23:59:42,2023-12-31 23:59:59,<NA>,False,<NA>,an,...,False,False,False,NaN,<NA>,NaN,None,None,None,None


In [14]:

tx_alarms_df

,alarm_id,data,first_occurrence,first_severity,last_occurrence,last_update,loc_name,make_prediction,ne_id,network,...,is_ack_or_notes,is_tt,is_abam_tt,tt_id,link_id,link_type,loc_name_a,loc_name_z,lp_id,lp_type
0,COS4P;86723842,2023-01-01 00:02:02,2022-12-31 23:45:36,critical,2022-12-31 23:55:53,2023-01-01 00:00:22,romaitnu,False,romaitnu/adsl/100,adsl,...,False,False,False,<NA>,corecoz,l12,NaN,NaN,None,None
1,COS3P;66481069,2023-01-01 00:02:02,2022-12-31 19:13:36,major,2022-12-31 21:58:05,2023-01-01 00:01:47,toriitbv,False,toriitbv/olth/203,adsl,...,False,False,False,<NA>,NaN,NaN,NaN,NaN,None,None
2,COS4P;86723883,2023-01-01 00:06:02,2023-01-01 00:02:07,major,2023-01-01 00:03:25,2023-01-01 00:04:47,napoitan,False,napoitan/olta/200,adsl,...,False,False,False,<NA>,NaN,NaN,NaN,NaN,None,None
3,COS4P;86723880,2023-01-01 00:06:02,2023-01-01 00:02:07,major,2023-01-01 00:03:25,2023-01-01 00:04:48,napoitan,False,napoitan/olta/200,adsl,...,False,False,False,<NA>,NaN,NaN,NaN,NaN,None,None
4,COS4P;86723881,2023-01-01 00:06:02,2023-01-01 00:02:07,major,2023-01-01 00:03:12,2023-01-01 00:04:49,napoitan,False,napoitan/olta/200,adsl,...,False,False,False,<NA>,NaN,NaN,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408772,EFMTX;65913ce9c23c110001552471,2024-01-01 00:00:03,2023-12-31 11:05:29,major,2023-12-31 23:59:31,2023-12-31 23:59:36,pstgitac,False,pstgitac/ptn3/1,ptn,...,False,False,False,<NA>,NaN,NaN,NaN,NaN,None,None
408773,EFMTX;65913e2cc23c110001552527,2024-01-01 00:00:03,2023-12-31 11:10:52,major,2023-12-31 23:59:09,2023-12-31 23:59:36,pstgitac,False,pstgitac/ptn3/1,ptn,...,False,False,False,<NA>,NaN,NaN,NaN,NaN,None,None
408774,EFMTX;6530d7a8eedb1500019c416e,2024-01-01 00:00:03,2023-10-19 09:15:52,warning,2023-12-31 23:59:14,2023-12-31 23:59:37,orviitaq,False,orviitaq/ptn3/1,ptn,...,True,False,False,<NA>,NaN,NaN,NaN,NaN,None,None
408775,EFMTX;658cd3bf975f510001572fd9,2024-01-01 00:00:03,2023-12-28 02:47:43,major,2023-12-31 23:59:29,2023-12-31 23:59:37,veneitab,False,veneitab/ptn3/1,ptn,...,True,False,False,<NA>,NaN,NaN,NaN,NaN,None,None


In [15]:
filtered_mob_alarms_df = mob_alarms_df.drop(columns=['lp_type', 'lp_id'])


MemoryError: Unable to allocate 215. MiB for an array with shape (28176444,) and data type object

MemoryError: Unable to allocate 215. MiB for an array with shape (1, 28176444) and data type object